<a href="https://colab.research.google.com/github/dalejo3000/pygraphs/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# INTEGRANTES GRUPO 1
# - Ismael Encadala
# - Francisco Marchán
# - Oscar Navarrete
# - David Romero

!pip install dash
!pip install dash-bootstrap-components # Install dash-bootstrap-components


import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

ruta_archivo = './clinical_analytics.csv'
datos = pd.read_csv(ruta_archivo)

datos['Hora Inicio Cita Convertida'] = pd.to_datetime(datos['Appt Start Time'], format='%Y-%m-%d %I:%M:%S %p', errors='coerce')
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H4("Análisis Operativo Clínico", className="text-info mb-4"),
            html.H3("Bienvenido al Dashboard de análisis de la operativa clínica", className="mb-3 text-white"),
            html.P(
                "Explore el volumen de pacientes por hora del día, tiempo de espera y puntuación de la atención. Haga clic en los gráficos para visualizar información más detallada.",
                className="text-white mb-4"
            ),
            html.Label("Clínica:", className="text-white"),
            dcc.Dropdown(
                id="filtro-clinica",
                options=[{"label": clinica, "value": clinica} for clinica in datos["Clinic Name"].unique()],
                value=datos["Clinic Name"].unique()[0],
                className="mb-4"
            ),
            html.Div([
                html.Label("Fecha:", className="text-white", style={"display": "inline-block", "margin-right": "10px"}),
                dcc.DatePickerRange(
                    id="filtro-fecha",
                    start_date=datos['Hora Inicio Cita Convertida'].min(),
                    end_date=datos['Hora Inicio Cita Convertida'].max(),
                    className="mb-4",
                    style={"display": "inline-block"}
                )
            ]),
            html.Label("Especialidad:", className="text-white"),
            dcc.Dropdown(
                id="filtro-especialidad",
                options=[],
                value=[],
                multi=True,
                className="mb-4"
            ),
            html.Div(id="mensaje-error", className="text-danger")
        ], width=4, className="bg-dark p-4 rounded"),

        dbc.Col([
            dbc.Tabs([
                dbc.Tab(dcc.Graph(id="grafico-mapa-calor", style={"margin-top": "20px"}), label="Volumen de pacientes por hora-día"),
                dbc.Tab(dcc.Graph(id="grafico-tendencia", style={"margin-top": "20px"}), label="Tiempo de espera"),
                dbc.Tab(dcc.Graph(id="grafico-puntajes", style={"margin-top": "20px"}), label="Puntajes de Atención"),
                dbc.Tab(dcc.Graph(id="grafico-caja-bigotes", style={"margin-top": "20px"}), label="Tiempo de espera vs puntaje de atención")
            ])
        ], width=8, style={"padding-top": "20px"})
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Pacientes Totales", className="card-title"),
                    html.H4(id="total-pacientes", className="card-text")
                ])
            ], color="primary", inverse=True),
        ], width=6),

        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H5("Tiempo de Espera Promedio", className="card-title"),
                    html.H4(id="espera-promedio", className="card-text")
                ])
            ], color="success", inverse=True),
        ], width=6)
    ])
], fluid=True)

@app.callback(
    Output("filtro-especialidad", "options"),
    Input("filtro-clinica", "value")
)
def actualizar_especialidades(clinica):
    especialidades_filtradas = datos[datos["Clinic Name"] == clinica]["Department"].unique()
    return [{"label": esp, "value": esp} for esp in especialidades_filtradas]

@app.callback(
    [
        Output("grafico-mapa-calor", "figure"),
        Output("grafico-tendencia", "figure"),
        Output("grafico-puntajes", "figure"),
        Output("grafico-caja-bigotes", "figure"),
        Output("total-pacientes", "children"),
        Output("espera-promedio", "children"),
        Output("mensaje-error", "children")
    ],
    [Input("filtro-clinica", "value"),
     Input("filtro-fecha", "start_date"),
     Input("filtro-fecha", "end_date"),
     Input("filtro-especialidad", "value")]
)
def actualizar_dashboard(clinica, fecha_inicio, fecha_fin, especialidades):
    datos_filtrados = datos[(datos["Clinic Name"] == clinica) &
                            (datos['Hora Inicio Cita Convertida'] >= fecha_inicio) &
                            (datos['Hora Inicio Cita Convertida'] <= fecha_fin)]

    if especialidades:
        datos_filtrados = datos_filtrados[datos_filtrados['Department'].isin(especialidades)]

    if datos_filtrados.empty:
        return ({}, {}, {}, {}, "No hay datos", "No hay datos", "No hay datos para las especialidades seleccionadas.")

    datos_filtrados['Dia Semana'] = datos_filtrados['Hora Inicio Cita Convertida'].dt.day_name()
    datos_filtrados['Hora'] = datos_filtrados['Hora Inicio Cita Convertida'].dt.hour
    mapa_calor = datos_filtrados.groupby(['Dia Semana', 'Hora']).size().reset_index(name='Volumen')
    grafico_mapa_calor = px.density_heatmap(mapa_calor, x='Hora', y='Dia Semana', z='Volumen',
                                            title='Volumen de Pacientes por Hora y Día',
                                            labels={'Hora': 'Hora', 'Dia Semana': 'Día', 'Volumen': 'Volumen'},
                                            color_continuous_scale='Viridis')

    tendencia = datos_filtrados.groupby(datos_filtrados['Hora Inicio Cita Convertida'].dt.date)['Wait Time Min'].mean().reset_index()
    tendencia.columns = ['Fecha', 'Tiempo de Espera Promedio']
    grafico_tendencia = px.line(tendencia, x='Fecha', y='Tiempo de Espera Promedio',
                                title='Evolución del Tiempo de Espera Promedio',
                                labels={'Fecha': 'Fecha', 'Tiempo de Espera Promedio': 'Tiempo de Espera (min)'})

    grafico_puntajes = px.histogram(datos_filtrados, x='Care Score', color='Care Score',
                                    title='Distribución de Puntajes de Atención',
                                    labels={'Care Score': 'Puntaje de Atención'},
                                    color_discrete_sequence=px.colors.sequential.Plasma)

    grafico_caja_bigotes = px.box(datos_filtrados, x='Care Score', y='Wait Time Min',
                                  title='Distribución de Tiempo de Espera por Puntaje de Atención',
                                  labels={'Care Score': 'Puntaje de Atención', 'Wait Time Min': 'Tiempo de Espera (min)'},
                                  color='Care Score', color_discrete_sequence=px.colors.sequential.Inferno)

    total_pacientes = len(datos_filtrados)
    espera_promedio = f"{datos_filtrados['Wait Time Min'].copy().mean():.2f} min"

    return (
        grafico_mapa_calor,
        grafico_tendencia,
        grafico_puntajes,
        grafico_caja_bigotes,
        total_pacientes,
        espera_promedio,
        ""
    )

if __name__ == "__main__":
    app.run_server(debug=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 1.7 MB/s eta 0:00:00


<IPython.core.display.Javascript object>